<a href="https://colab.research.google.com/github/wesslen/llm-examples/blob/main/notebooks/dspy_credit_react_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!uv pip install --system dspy==3.0.1

In [1]:
import dspy

dspy.__version__

'3.0.1'

## LLM Setup

In [2]:
from google.colab import userdata

lm = dspy.LM("gemini/gemini-2.5-flash-lite", api_key = userdata.get('GOOGLE_API_KEY'))
dspy.configure(lm = lm)

## Schema

In [5]:
from pydantic import BaseModel, Field
from typing import Optional, List, Literal
from datetime import datetime


class Date(BaseModel):
    # Custom date class for better LLM compatibility
    year: int
    month: int
    day: int


class FinancialStatement(BaseModel):
    """Financial statement data for a company"""
    company_id: str
    period_end: Date
    total_revenue: float = Field(ge=0, description="Total revenue for the period")
    net_income: float = Field(description="Net income (can be negative)")
    total_assets: float = Field(gt=0, description="Total assets (must be positive)")
    total_liabilities: float = Field(ge=0, description="Total liabilities")
    shareholders_equity: float = Field(description="Shareholders equity")
    current_assets: float = Field(ge=0, description="Current assets")
    current_liabilities: float = Field(ge=0, description="Current liabilities")
    cash_and_equivalents: float = Field(ge=0, description="Cash and cash equivalents")
    total_debt: float = Field(ge=0, description="Total debt")
    ebitda: float = Field(description="Earnings before interest, taxes, depreciation, and amortization")
    interest_expense: float = Field(ge=0, description="Interest expense")
    working_capital: float = Field(description="Working capital (current assets - current liabilities)")


class CreditProfile(BaseModel):
    """Credit profile for a borrower"""
    company_id: str
    credit_rating: str = Field(description="Credit rating (e.g., AAA, AA, A, BBB, etc.)")
    industry: str = Field(description="Industry classification")
    years_in_business: int = Field(ge=0, description="Number of years in business")
    loan_amount: float = Field(gt=0, description="Requested loan amount")
    annual_revenue: float = Field(ge=0, description="Annual revenue")


class RatioMetrics(BaseModel):
    """Collection of calculated financial ratios"""
    debt_to_equity_ratio: float = Field(description="Total debt divided by shareholders equity")
    current_ratio: float = Field(description="Current assets divided by current liabilities")
    interest_coverage_ratio: float = Field(description="EBITDA divided by interest expense")
    return_on_assets: float = Field(description="Net income divided by total assets")
    revenue_growth_rate: Optional[float] = Field(default=None, description="Year-over-year revenue growth rate")


class AltmanZScore(BaseModel):
    """Altman Z-Score calculation results"""
    z_score: float = Field(description="Calculated Altman Z-Score")
    risk_category: Literal['low_risk', 'moderate_risk', 'high_risk'] = Field(
        description="Risk category based on Z-Score thresholds"
    )


class WorkingCapitalTrend(BaseModel):
    """Working capital trend analysis results"""
    average_growth: float = Field(description="Average working capital growth rate")
    trend_direction: Literal['improving', 'declining'] = Field(
        description="Overall trend direction"
    )
    periods_analyzed: int = Field(ge=1, description="Number of periods included in analysis")


class CreditRiskReport(BaseModel):
    """Comprehensive credit risk analysis report"""
    report_id: str
    company_id: str
    analysis_date: datetime
    period_analyzed: int = Field(ge=1, description="Number of years analyzed")
    debt_to_equity_ratio: float
    current_ratio: float
    interest_coverage_ratio: float
    return_on_assets: float
    revenue_growth_rate: Optional[float] = None
    altman_z_score: AltmanZScore
    working_capital_trend: WorkingCapitalTrend
    overall_risk_rating: Literal['low_risk', 'low_moderate_risk', 'moderate_risk', 'high_risk'] = Field(
        description="Overall risk assessment based on multiple factors"
    )


class LoanApplication(BaseModel):
    """Loan application combining credit profile and financial data"""
    application_id: str
    credit_profile: CreditProfile
    financial_statements: List[FinancialStatement] = Field(
        min_items=1, description="Historical financial statements (most recent first)"
    )
    requested_terms: Optional[dict] = Field(
        default=None, description="Requested loan terms (interest rate, duration, etc.)"
    )


class RiskAssessmentRequest(BaseModel):
    """Request for risk assessment analysis"""
    company_id: str
    analysis_period: int = Field(default=3, ge=1, le=10, description="Number of years to analyze")
    include_trends: bool = Field(default=True, description="Whether to include trend analysis")
    custom_weights: Optional[dict] = Field(
        default=None, description="Custom weights for risk factors"
    )


class CreditDecision(BaseModel):
    """Final credit decision based on risk assessment"""
    decision_id: str
    application_id: str
    report_id: str
    decision: Literal['approved', 'rejected', 'conditional'] = Field(
        description="Final credit decision"
    )
    approved_amount: Optional[float] = Field(
        default=None, ge=0, description="Approved loan amount (if approved or conditional)"
    )
    interest_rate: Optional[float] = Field(
        default=None, ge=0, description="Approved interest rate"
    )
    conditions: Optional[List[str]] = Field(
        default=None, description="Conditions for approval (if conditional)"
    )
    decision_date: datetime
    decision_rationale: str = Field(description="Explanation for the decision")


class IndustryBenchmark(BaseModel):
    """Industry benchmark data for comparison"""
    industry: str
    median_debt_to_equity: float
    median_current_ratio: float
    median_interest_coverage: float
    median_roa: float
    median_revenue_growth: float
    data_source: str = Field(description="Source of benchmark data")
    last_updated: Date


class CreditMonitoring(BaseModel):
    """Ongoing credit monitoring for existing loans"""
    monitoring_id: str
    company_id: str
    loan_id: str
    current_risk_rating: Literal['low_risk', 'low_moderate_risk', 'moderate_risk', 'high_risk']
    previous_risk_rating: Optional[str] = None
    rating_change_date: datetime
    next_review_date: Date
    monitoring_frequency: Literal['monthly', 'quarterly', 'semi_annual', 'annual']
    alerts: Optional[List[str]] = Field(
        default=None, description="Any risk alerts or covenant violations"
    )


class CreditTicket(BaseModel):
    """Support ticket for credit-related issues"""
    ticket_id: str
    borrower_request: str = Field(description="Description of the borrower's request or issue")
    credit_profile: CreditProfile
    priority: Literal['low', 'medium', 'high', 'urgent'] = Field(default='medium')
    category: Literal['application', 'existing_loan', 'payment_issue', 'modification_request', 'other'] = Field(
        description="Category of the credit ticket"
    )
    created_date: datetime
    assigned_to: Optional[str] = Field(default=None, description="Credit analyst assigned to the ticket")

## Data

In [6]:
# Credit Risk System Databases
from datetime import datetime

# Credit profiles database - indexed by borrower name
credit_profile_database = {
    "TechCorp Inc": CreditProfile(
        company_id="TC001",
        credit_rating="BBB+",
        industry="Technology",
        years_in_business=8,
        loan_amount=2500000.00,
        annual_revenue=15000000.00
    ),
    "Manufacturing Solutions LLC": CreditProfile(
        company_id="MS002",
        credit_rating="A-",
        industry="Manufacturing",
        years_in_business=15,
        loan_amount=5000000.00,
        annual_revenue=35000000.00
    ),
    "Green Energy Partners": CreditProfile(
        company_id="GE003",
        credit_rating="BB",
        industry="Renewable Energy",
        years_in_business=5,
        loan_amount=8000000.00,
        annual_revenue=12000000.00
    ),
    "Regional Retail Group": CreditProfile(
        company_id="RR004",
        credit_rating="BBB",
        industry="Retail",
        years_in_business=22,
        loan_amount=3000000.00,
        annual_revenue=28000000.00
    ),
    "Healthcare Innovations": CreditProfile(
        company_id="HI005",
        credit_rating="A",
        industry="Healthcare",
        years_in_business=12,
        loan_amount=4500000.00,
        annual_revenue=25000000.00
    ),
}

# Financial statements database - indexed by company_id, contains list of statements
financial_data_database = {
    "TC001": [
        # 2024 - Most recent
        FinancialStatement(
            company_id="TC001",
            period_end=Date(year=2024, month=12, day=31),
            total_revenue=15000000.00,
            net_income=1800000.00,
            total_assets=8500000.00,
            total_liabilities=3200000.00,
            shareholders_equity=5300000.00,
            current_assets=3500000.00,
            current_liabilities=1200000.00,
            cash_and_equivalents=1500000.00,
            total_debt=2800000.00,
            ebitda=2400000.00,
            interest_expense=280000.00,
            working_capital=2300000.00
        ),
        # 2023
        FinancialStatement(
            company_id="TC001",
            period_end=Date(year=2023, month=12, day=31),
            total_revenue=12000000.00,
            net_income=1200000.00,
            total_assets=7200000.00,
            total_liabilities=2800000.00,
            shareholders_equity=4400000.00,
            current_assets=2800000.00,
            current_liabilities=1000000.00,
            cash_and_equivalents=1200000.00,
            total_debt=2400000.00,
            ebitda=1800000.00,
            interest_expense=240000.00,
            working_capital=1800000.00
        ),
        # 2022
        FinancialStatement(
            company_id="TC001",
            period_end=Date(year=2022, month=12, day=31),
            total_revenue=9500000.00,
            net_income=800000.00,
            total_assets=6000000.00,
            total_liabilities=2200000.00,
            shareholders_equity=3800000.00,
            current_assets=2200000.00,
            current_liabilities=800000.00,
            cash_and_equivalents=900000.00,
            total_debt=1800000.00,
            ebitda=1400000.00,
            interest_expense=180000.00,
            working_capital=1400000.00
        )
    ],
    "MS002": [
        # 2024
        FinancialStatement(
            company_id="MS002",
            period_end=Date(year=2024, month=12, day=31),
            total_revenue=35000000.00,
            net_income=3200000.00,
            total_assets=28000000.00,
            total_liabilities=12000000.00,
            shareholders_equity=16000000.00,
            current_assets=8500000.00,
            current_liabilities=3200000.00,
            cash_and_equivalents=2800000.00,
            total_debt=9500000.00,
            ebitda=5800000.00,
            interest_expense=475000.00,
            working_capital=5300000.00
        ),
        # 2023
        FinancialStatement(
            company_id="MS002",
            period_end=Date(year=2023, month=12, day=31),
            total_revenue=32000000.00,
            net_income=2800000.00,
            total_assets=25500000.00,
            total_liabilities=11200000.00,
            shareholders_equity=14300000.00,
            current_assets=7800000.00,
            current_liabilities=2900000.00,
            cash_and_equivalents=2400000.00,
            total_debt=8800000.00,
            ebitda=5200000.00,
            interest_expense=440000.00,
            working_capital=4900000.00
        ),
        # 2022
        FinancialStatement(
            company_id="MS002",
            period_end=Date(year=2022, month=12, day=31),
            total_revenue=29000000.00,
            net_income=2200000.00,
            total_assets=23000000.00,
            total_liabilities=10500000.00,
            shareholders_equity=12500000.00,
            current_assets=7000000.00,
            current_liabilities=2600000.00,
            cash_and_equivalents=2000000.00,
            total_debt=8000000.00,
            ebitda=4600000.00,
            interest_expense=400000.00,
            working_capital=4400000.00
        )
    ],
    "GE003": [
        # 2024
        FinancialStatement(
            company_id="GE003",
            period_end=Date(year=2024, month=12, day=31),
            total_revenue=12000000.00,
            net_income=600000.00,
            total_assets=15000000.00,
            total_liabilities=9200000.00,
            shareholders_equity=5800000.00,
            current_assets=4200000.00,
            current_liabilities=2800000.00,
            cash_and_equivalents=1800000.00,
            total_debt=7500000.00,
            ebitda=1800000.00,
            interest_expense=525000.00,
            working_capital=1400000.00
        ),
        # 2023
        FinancialStatement(
            company_id="GE003",
            period_end=Date(year=2023, month=12, day=31),
            total_revenue=8500000.00,
            net_income=200000.00,
            total_assets=12500000.00,
            total_liabilities=8000000.00,
            shareholders_equity=4500000.00,
            current_assets=3500000.00,
            current_liabilities=2200000.00,
            cash_and_equivalents=1200000.00,
            total_debt=6500000.00,
            ebitda=1200000.00,
            interest_expense=455000.00,
            working_capital=1300000.00
        ),
        # 2022
        FinancialStatement(
            company_id="GE003",
            period_end=Date(year=2022, month=12, day=31),
            total_revenue=6000000.00,
            net_income=-100000.00,
            total_assets=10000000.00,
            total_liabilities=6800000.00,
            shareholders_equity=3200000.00,
            current_assets=2800000.00,
            current_liabilities=1800000.00,
            cash_and_equivalents=800000.00,
            total_debt=5200000.00,
            ebitda=800000.00,
            interest_expense=364000.00,
            working_capital=1000000.00
        )
    ],
    "RR004": [
        # 2024
        FinancialStatement(
            company_id="RR004",
            period_end=Date(year=2024, month=12, day=31),
            total_revenue=28000000.00,
            net_income=1400000.00,
            total_assets=18500000.00,
            total_liabilities=7800000.00,
            shareholders_equity=10700000.00,
            current_assets=6500000.00,
            current_liabilities=2400000.00,
            cash_and_equivalents=2200000.00,
            total_debt=5500000.00,
            ebitda=3200000.00,
            interest_expense=385000.00,
            working_capital=4100000.00
        ),
        # 2023
        FinancialStatement(
            company_id="RR004",
            period_end=Date(year=2023, month=12, day=31),
            total_revenue=26500000.00,
            net_income=1200000.00,
            total_assets=17200000.00,
            total_liabilities=7200000.00,
            shareholders_equity=10000000.00,
            current_assets=6000000.00,
            current_liabilities=2200000.00,
            cash_and_equivalents=1900000.00,
            total_debt=5000000.00,
            ebitda=2900000.00,
            interest_expense=350000.00,
            working_capital=3800000.00
        ),
        # 2022
        FinancialStatement(
            company_id="RR004",
            period_end=Date(year=2022, month=12, day=31),
            total_revenue=25000000.00,
            net_income=1000000.00,
            total_assets=16000000.00,
            total_liabilities=6800000.00,
            shareholders_equity=9200000.00,
            current_assets=5500000.00,
            current_liabilities=2000000.00,
            cash_and_equivalents=1600000.00,
            total_debt=4500000.00,
            ebitda=2600000.00,
            interest_expense=315000.00,
            working_capital=3500000.00
        )
    ],
    "HI005": [
        # 2024
        FinancialStatement(
            company_id="HI005",
            period_end=Date(year=2024, month=12, day=31),
            total_revenue=25000000.00,
            net_income=2750000.00,
            total_assets=22000000.00,
            total_liabilities=8500000.00,
            shareholders_equity=13500000.00,
            current_assets=7800000.00,
            current_liabilities=2600000.00,
            cash_and_equivalents=3200000.00,
            total_debt=6200000.00,
            ebitda=4200000.00,
            interest_expense=434000.00,
            working_capital=5200000.00
        ),
        # 2023
        FinancialStatement(
            company_id="HI005",
            period_end=Date(year=2023, month=12, day=31),
            total_revenue=22000000.00,
            net_income=2200000.00,
            total_assets=19500000.00,
            total_liabilities=7800000.00,
            shareholders_equity=11700000.00,
            current_assets=6800000.00,
            current_liabilities=2200000.00,
            cash_and_equivalents=2600000.00,
            total_debt=5500000.00,
            ebitda=3600000.00,
            interest_expense=385000.00,
            working_capital=4600000.00
        ),
        # 2022
        FinancialStatement(
            company_id="HI005",
            period_end=Date(year=2022, month=12, day=31),
            total_revenue=19000000.00,
            net_income=1800000.00,
            total_assets=17000000.00,
            total_liabilities=7000000.00,
            shareholders_equity=10000000.00,
            current_assets=5800000.00,
            current_liabilities=1900000.00,
            cash_and_equivalents=2200000.00,
            total_debt=4800000.00,
            ebitda=3000000.00,
            interest_expense=336000.00,
            working_capital=3900000.00
        )
    ]
}

# Industry benchmarks database
industry_benchmark_database = {
    "Technology": IndustryBenchmark(
        industry="Technology",
        median_debt_to_equity=0.8,
        median_current_ratio=2.1,
        median_interest_coverage=8.5,
        median_roa=0.12,
        median_revenue_growth=0.15,
        data_source="Industry Research Institute",
        last_updated=Date(year=2024, month=6, day=30)
    ),
    "Manufacturing": IndustryBenchmark(
        industry="Manufacturing",
        median_debt_to_equity=1.2,
        median_current_ratio=1.8,
        median_interest_coverage=6.2,
        median_roa=0.08,
        median_revenue_growth=0.06,
        data_source="Manufacturing Analytics Corp",
        last_updated=Date(year=2024, month=6, day=30)
    ),
    "Renewable Energy": IndustryBenchmark(
        industry="Renewable Energy",
        median_debt_to_equity=1.8,
        median_current_ratio=1.5,
        median_interest_coverage=3.2,
        median_roa=0.04,
        median_revenue_growth=0.25,
        data_source="Energy Sector Analysis",
        last_updated=Date(year=2024, month=6, day=30)
    ),
    "Retail": IndustryBenchmark(
        industry="Retail",
        median_debt_to_equity=0.9,
        median_current_ratio=1.6,
        median_interest_coverage=5.8,
        median_roa=0.06,
        median_revenue_growth=0.04,
        data_source="Retail Industry Report",
        last_updated=Date(year=2024, month=6, day=30)
    ),
    "Healthcare": IndustryBenchmark(
        industry="Healthcare",
        median_debt_to_equity=0.7,
        median_current_ratio=2.3,
        median_interest_coverage=9.1,
        median_roa=0.14,
        median_revenue_growth=0.08,
        data_source="Healthcare Finance Institute",
        last_updated=Date(year=2024, month=6, day=30)
    )
}

# Empty databases for runtime operations (similar to your example)
loan_application_database = {}
risk_assessment_database = {}
credit_decision_database = {}
credit_monitoring_database = {}
credit_ticket_database = {}

# Sample loan applications database with some initial data
loan_application_database = {
    "APP001": LoanApplication(
        application_id="APP001",
        credit_profile=credit_profile_database["TechCorp Inc"],
        financial_statements=financial_data_database["TC001"],
        requested_terms={
            "loan_duration_months": 60,
            "requested_rate": 0.065,
            "collateral_type": "Equipment"
        }
    ),
    "APP002": LoanApplication(
        application_id="APP002",
        credit_profile=credit_profile_database["Manufacturing Solutions LLC"],
        financial_statements=financial_data_database["MS002"],
        requested_terms={
            "loan_duration_months": 84,
            "requested_rate": 0.055,
            "collateral_type": "Real Estate"
        }
    )
}

## Tools

In [61]:
import random
import string
from typing import Dict, List, Optional
from dataclasses import dataclass
from datetime import datetime, date

# Using Pydantic models directly is generally preferred for validation and data handling,
# but if dataclasses are needed for specific functions, ensure they match the Pydantic definitions.
# In this case, the functions were written to accept FinancialStatement objects, so matching the structure is necessary.
@dataclass
class FinancialStatement:
    """Financial statement data for a company"""
    company_id: str
    period_end: Date # Changed from date to Date to match Pydantic model
    total_revenue: float
    net_income: float
    total_assets: float
    total_liabilities: float
    shareholders_equity: float
    current_assets: float
    current_liabilities: float
    cash_and_equivalents: float
    total_debt: float
    ebitda: float
    interest_expense: float
    working_capital: float


@dataclass
class CreditProfile:
    """Credit profile for a borrower"""
    company_id: str
    credit_rating: str
    industry: str
    years_in_business: int
    loan_amount: float
    annual_revenue: float


def fetch_financial_statements(company_id: str, years: int = 3):
    """Fetch historical financial statements for a company - with sorting fix"""
    statements = financial_data_database.get(company_id, [])

    # Sort by period_end date using a custom key that converts to tuple
    sorted_statements = sorted(statements,
                              key=lambda x: (x.period_end.year, x.period_end.month, x.period_end.day),
                              reverse=True)

    if len(sorted_statements) < years:
        raise ValueError(f"Insufficient financial data: only {len(sorted_statements)} years available, {years} requested")

    return sorted_statements[:years]


def calculate_debt_to_equity_ratio(financial_statement: FinancialStatement):
    """Calculate debt-to-equity ratio for credit risk assessment"""
    if financial_statement.shareholders_equity <= 0:
        raise ValueError("Invalid shareholders equity: cannot calculate debt-to-equity ratio")

    return financial_statement.total_debt / financial_statement.shareholders_equity


def calculate_current_ratio(financial_statement: FinancialStatement):
    """Calculate current ratio (liquidity measure)"""
    if financial_statement.current_liabilities <= 0:
        raise ValueError("Invalid current liabilities: cannot calculate current ratio")

    return financial_statement.current_assets / financial_statement.current_liabilities


def calculate_interest_coverage_ratio(financial_statement: FinancialStatement):
    """Calculate interest coverage ratio (ability to service debt)"""
    if financial_statement.interest_expense <= 0:
        raise ValueError("Invalid interest expense: cannot calculate interest coverage ratio")

    return financial_statement.ebitda / financial_statement.interest_expense


def calculate_debt_service_coverage_ratio(net_operating_income: float, total_debt_service: float):
    """Calculate debt service coverage ratio"""
    if total_debt_service <= 0:
        raise ValueError("Invalid total debt service: must be greater than zero")

    return net_operating_income / total_debt_service


def calculate_return_on_assets(financial_statement: FinancialStatement):
    """Calculate return on assets (profitability measure)"""
    if financial_statement.total_assets <= 0:
        raise ValueError("Invalid total assets: cannot calculate return on assets")

    return financial_statement.net_income / financial_statement.total_assets


def calculate_revenue_growth_rate(current_revenue: float, prior_revenue: float):
    """Calculate year-over-year revenue growth rate"""
    if prior_revenue <= 0:
        # The original error message was using an f-string but wasn't correctly formatted.
        raise ValueError(f"Invalid prior year revenue: cannot calculate growth rate (prior revenue was {prior_revenue})")

    return (current_revenue - prior_revenue) / prior_revenue


def assess_working_capital_trend(financial_statements):
    """Assess working capital trend over multiple periods"""
    if len(financial_statements) < 2:
        raise ValueError("Need at least 2 periods to assess trend")

    # Sort by date (oldest first for trend analysis) using tuple comparison
    sorted_statements = sorted(financial_statements,
                              key=lambda x: (x.period_end.year, x.period_end.month, x.period_end.day))

    trends = []
    for i in range(1, len(sorted_statements)):
        current_wc = sorted_statements[i].working_capital
        prior_wc = sorted_statements[i-1].working_capital

        if prior_wc != 0:
            growth_rate = (current_wc - prior_wc) / abs(prior_wc)
            trends.append(growth_rate)

    return {
        'average_growth': sum(trends) / len(trends) if trends else 0,
        'trend_direction': 'improving' if sum(trends) > 0 else 'declining',
        'periods_analyzed': len(trends)
    }


def calculate_altman_z_score(financial_statement: FinancialStatement):
    """Calculate Altman Z-Score for bankruptcy prediction"""
    if financial_statement.total_assets <= 0:
        raise ValueError("Invalid total assets: cannot calculate Z-Score")

    # Altman Z-Score components
    wc_ta = financial_statement.working_capital / financial_statement.total_assets
    re_ta = (financial_statement.shareholders_equity) / financial_statement.total_assets # Corrected Retained Earnings calculation
    ebit_ta = financial_statement.ebitda / financial_statement.total_assets  # Using EBITDA as proxy for EBIT
    me_tl = financial_statement.shareholders_equity / financial_statement.total_liabilities if financial_statement.total_liabilities > 0 else 0
    sales_ta = financial_statement.total_revenue / financial_statement.total_assets

    # Altman Z-Score formula
    z_score = (1.2 * wc_ta) + (1.4 * re_ta) + (3.3 * ebit_ta) + (0.6 * me_tl) + (1.0 * sales_ta)

    return {
        'z_score': z_score,
        'risk_category': _categorize_z_score(z_score)
    }


def _categorize_z_score(z_score: float):
    """Categorize Z-Score into risk levels"""
    # Using standard thresholds for private companies (Type B)
    if z_score > 2.6:
        return 'low_risk'
    elif z_score >= 1.1:
        return 'moderate_risk'
    else:
        return 'high_risk'


def generate_credit_risk_report(company_id: str, analysis_period: int = 3):
    """Generate comprehensive credit risk analysis report"""
    try:
        statements = fetch_financial_statements(company_id, analysis_period)
        latest_statement = statements[0]

        # Calculate key ratios
        debt_equity = calculate_debt_to_equity_ratio(latest_statement)
        current_ratio = calculate_current_ratio(latest_statement)
        interest_coverage = calculate_interest_coverage_ratio(latest_statement)
        roa = calculate_return_on_assets(latest_statement)
        altman_score = calculate_altman_z_score(latest_statement)
        wc_trend = assess_working_capital_trend(statements)

        # Calculate revenue growth if multiple periods available
        revenue_growth = None
        if len(statements) >= 2:
            revenue_growth = calculate_revenue_growth_rate(
                statements[0].total_revenue,
                statements[1].total_revenue
            )

        report_id = _generate_report_id()

        risk_report = {
            'report_id': report_id,
            'company_id': company_id,
            'analysis_date': datetime.now(),
            'period_analyzed': analysis_period,
            'debt_to_equity_ratio': debt_equity,
            'current_ratio': current_ratio,
            'interest_coverage_ratio': interest_coverage,
            'return_on_assets': roa,
            'revenue_growth_rate': revenue_growth,
            'altman_z_score': altman_score,
            'working_capital_trend': wc_trend,
            'overall_risk_rating': _determine_overall_risk(debt_equity, current_ratio, interest_coverage, altman_score['z_score'])
        }

        risk_assessment_database[report_id] = risk_report
        return report_id, risk_report

    except ValueError as e:
        raise ValueError(f"Cannot generate credit risk report: {str(e)}")


def _generate_report_id(length=10):
    """Generate unique report ID"""
    chars = string.ascii_uppercase + string.digits
    return "CR" + "".join(random.choices(chars, k=length))


def _determine_overall_risk(debt_equity: float, current_ratio: float, interest_coverage: float, z_score: float):
    """Determine overall risk rating based on multiple factors"""
    risk_score = 0

    # Debt-to-equity scoring (higher is riskier)
    if debt_equity > 2.0:
        risk_score += 3
    elif debt_equity > 1.0:
        risk_score += 2
    elif debt_equity > 0.5:
        risk_score += 1

    # Current ratio scoring (lower is riskier)
    if current_ratio < 1.0:
        risk_score += 3
    elif current_ratio < 1.5:
        risk_score += 2
    elif current_ratio < 2.0:
        risk_score += 1

    # Interest coverage scoring (lower is riskier, handle infinite case)
    if interest_coverage < 1.5:
        risk_score += 3
    elif interest_coverage < 2.5:
        risk_score += 2
    elif interest_coverage < 4.0:
        risk_score += 1
    # If interest_coverage is inf, no score is added, implying lower risk from this factor

    # Z-Score scoring (using Type B thresholds)
    if z_score < 1.1:
        risk_score += 3
    elif z_score < 2.6:
        risk_score += 1

    # Determine final rating
    if risk_score >= 7: # Adjusted threshold based on the potential range of scores
        return 'high_risk'
    elif risk_score >= 4:
        return 'moderate_risk'
    elif risk_score >= 1:
        return 'low_moderate_risk'
    else:
        return 'low_risk'


def fetch_credit_profile(company_id: str):
    """Fetch credit profile information from database"""
    profile = credit_profile_database.get(company_id)
    if not profile:
        raise ValueError("Credit profile not found for the specified borrower")
    return profile


def fetch_risk_assessment(report_id: str):
    """Fetch a previously generated risk assessment report"""
    assessment = risk_assessment_database.get(report_id)
    if not assessment:
        raise ValueError("Risk assessment report not found")
    return assessment

# Define the tools that the DSPy agent can use.
# These should correspond to the functions defined above that perform specific actions.
# The agent will use these tools based on the user's request.
# We need to define the tools here as simple functions or objects that can be called.
# For the DSPy ReAct agent, these tools should ideally be callable functions.
# We will create simple wrapper functions for the existing logic to serve as tools.


def analyze_company_creditworthiness(company_id: str, analysis_years: int = 3):
    """Perform comprehensive creditworthiness analysis for a company"""
    try:
        # Find credit profile by company_id
        credit_profile = None
        company_name = None
        for name, profile in credit_profile_database.items():
            if profile.company_id == company_id:
                credit_profile = profile
                company_name = name
                break

        if not credit_profile:
            raise ValueError(f"Company with ID '{company_id}' not found in credit database")

        # Generate comprehensive risk report
        report_id, risk_report = generate_credit_risk_report(company_id, analysis_years)

        # Get industry benchmark for comparison
        industry = credit_profile.industry
        benchmark = industry_benchmark_database.get(industry)

        return {
            'report_id': report_id,
            'company_id': company_id,
            'company_name': company_name,
            'credit_profile': credit_profile,
            'risk_report': risk_report,
            'industry_benchmark': benchmark
        }
    except Exception as e:
        raise ValueError(f"Cannot analyze creditworthiness: {str(e)}")


def compare_to_industry_benchmark(company_id: str):
    """Compares a company's key financial metrics to industry benchmarks."""
    try:
        profile = fetch_credit_profile(company_id)
        latest_statement = fetch_financial_statements(company_id, 1)[0]
        industry = profile.industry
        benchmark = industry_benchmark_database.get(industry)

        if not benchmark:
            return f"No industry benchmark data found for {industry}."

        debt_equity = calculate_debt_to_equity_ratio(latest_statement)
        current_ratio = calculate_current_ratio(latest_statement)
        interest_coverage = calculate_interest_coverage_ratio(latest_statement) if latest_statement.interest_expense > 0 else float('inf')
        roa = calculate_return_on_assets(latest_statement)
        # Revenue growth requires at least two periods, handle if not available
        revenue_growth = None
        try:
             statements = fetch_financial_statements(company_id, 2)
             revenue_growth = calculate_revenue_growth_rate(statements[0].total_revenue, statements[1].total_revenue)
        except (ValueError, IndexError):
             revenue_growth = None


        comparison_output = f"Comparison for {company_id} ({industry}):\n"
        comparison_output += f"- Debt-to-Equity Ratio: Company = {debt_equity:.2f}, Industry Median = {benchmark.median_debt_to_equity:.2f}\n"
        comparison_output += f"- Current Ratio: Company = {current_ratio:.2f}, Industry Median = {benchmark.median_current_ratio:.2f}\n"
        comparison_output += f"- Interest Coverage Ratio: Company = {interest_coverage:.2f}, Industry Median = {benchmark.median_interest_coverage:.2f}\n"
        comparison_output += f"- Return on Assets (ROA): Company = {roa:.2f}, Industry Median = {benchmark.median_roa:.2f}\n"
        if revenue_growth is not None:
             comparison_output += f"- Revenue Growth Rate (YoY): Company = {revenue_growth:.2%}, Industry Median = {benchmark.median_revenue_growth:.2%}\n"
        else:
             comparison_output += "- Revenue Growth Rate (YoY): Insufficient data for company.\n"

        return comparison_output

    except ValueError as e:
        return f"Cannot compare to industry benchmark: {str(e)}"


# Placeholder tools for other functionalities - these would need to be implemented based on the Pydantic models
def process_loan_application(application_id: str):
    """Processes a loan application, performs risk assessment, and makes a credit decision."""
    # This would involve fetching the loan application, running risk analysis, and generating a decision.
    return f"Processing loan application {application_id}. This functionality is not yet fully implemented."


def monitor_existing_credit(loan_id: str):
    """Monitors an existing credit relationship for changes in risk profile."""
    # This would involve checking for updated financial data, covenant compliance, etc.
    return f"Monitoring credit for loan {loan_id}. This functionality is not yet fully implemented."


def calculate_financial_ratios(company_id: str, year: int):
    """Calculates key financial ratios for a specific company and year."""
    try:
        statements = fetch_financial_statements(company_id, years=5) # Fetch enough years to find the specific year
        statement_for_year = None
        for stmt in statements:
            if stmt.period_end.year == year:
                statement_for_year = stmt
                break

        if not statement_for_year:
            return f"Financial statement for {year} not found for {company_id}."

        debt_equity = calculate_debt_to_equity_ratio(statement_for_year)
        current_ratio = calculate_current_ratio(statement_for_year)
        interest_coverage = calculate_interest_coverage_ratio(statement_for_year) if statement_for_year.interest_expense > 0 else float('inf')
        roa = calculate_return_on_assets(statement_for_year)
        altman_score_result = calculate_altman_z_score(statement_for_year)

        ratios_output = f"Financial Ratios for {company_id} ({year}):\n"
        ratios_output += f"- Debt-to-Equity Ratio: {debt_equity:.2f}\n"
        ratios_output += f"- Current Ratio: {current_ratio:.2f}\n"
        ratios_output += f"- Interest Coverage Ratio: {interest_coverage:.2f}\n"
        ratios_output += f"- Return on Assets (ROA): {roa:.2f}\n"
        ratios_output += f"- Altman Z-Score: {altman_score_result['z_score']:.2f} ({altman_score_result['risk_category']})\n"

        return ratios_output

    except ValueError as e:
        return f"Cannot calculate financial ratios: {str(e)}"

def get_company_id_from_name(company_name: str):
    """Convert company name to company ID for use with other functions"""
    if company_name not in credit_profile_database:
        # Try fuzzy matching in case of slight variations
        available_companies = list(credit_profile_database.keys())
        raise ValueError(f"Company '{company_name}' not found. Available companies: {available_companies}")

    credit_profile = credit_profile_database[company_name]
    return credit_profile.company_id


def get_credit_profile_info(company_id: str):
    """Retrieves the credit profile information for a borrower."""
    try:
        profile = fetch_credit_profile(company_id)
        profile_info = f"Credit Profile for Borrower ID {company_id}:\n"
        profile_info += f"- Credit Rating: {profile.credit_rating}\n"
        profile_info += f"- Industry: {profile.industry}\n"
        profile_info += f"- Years in Business: {profile.years_in_business}\n"
        profile_info += f"- Requested Loan Amount: {profile.loan_amount:,.2f}\n"
        profile_info += f"- Annual Revenue: {profile.annual_revenue:,.2f}\n"
        return profile_info
    except ValueError as e:
        return f"Cannot retrieve credit profile: {str(e)}"


def file_credit_ticket(company_id: str, issue_description: str, priority: str = 'medium', category: str = 'other'):
    """Files a support ticket for a credit-related issue."""
    # This would involve creating a CreditTicket object and adding it to the credit_ticket_database.
    ticket_id = "TICKET" + "".join(random.choices(string.ascii_uppercase + string.digits, k=8))
    try:
        # Fetch credit profile to include in the ticket
        profile = fetch_credit_profile(company_id)
        new_ticket = {
            "ticket_id": ticket_id,
            "borrower_request": issue_description,
            "credit_profile": profile,
            "priority": priority,
            "category": category,
            "created_date": datetime.now(),
            "assigned_to": None # Initially unassigned
        }
        credit_ticket_database[ticket_id] = new_ticket
        return f"Credit ticket filed successfully. Ticket ID: {ticket_id}"
    except ValueError as e:
        return f"Cannot file credit ticket: {str(e)}"

# The fetch_financial_statements, fetch_credit_profile, and fetch_risk_assessment
# functions are helper functions used internally by the tools, they don't need to be exposed as tools themselves
# unless there's a specific reason for the agent to directly access raw data.
# For now, we will keep them as internal helpers for the tools.

## ReAct Agent

In [63]:
from typing import List, Dict, Optional
import random
import string
from datetime import datetime, date


class DSPyCreditRiskAnalyst(dspy.Signature):
    """You are a commercial credit risk analyst that helps assess creditworthiness and manage loan applications.

    You are given a list of tools to handle credit analysis requests, and you should decide the right tool to use
    in order to fulfill the request. You can analyze financial statements, calculate risk metrics, generate
    comprehensive risk reports, and make credit recommendations.

    Common tasks include:
    - Get company_id from a company's name provided by the user
    - Analyzing a company's financial health and creditworthiness by its company_id
    - Calculating key financial ratios (debt-to-equity, current ratio, interest coverage, etc.)
    - Generating comprehensive credit risk reports with recommendations
    - Comparing companies to industry benchmarks
    - Assessing loan applications and making credit decisions
    - Monitoring existing credit relationships
    - Filing tickets for complex credit issues that require specialist review
    """

    user_request: str = dspy.InputField()
    process_result: str = dspy.OutputField(
        desc=(
            "Detailed analysis result that summarizes the credit assessment findings, key financial metrics, "
            "risk ratings, and actionable recommendations. Include specific numbers, ratios, and risk categories. "
            "Provide report IDs, decision outcomes, or ticket numbers when applicable."
        )
    )

# Create the DSPy ReAct Agent
# Ensure the tools listed here match the functions intended for the agent to call.
agent = dspy.ReAct(
    DSPyCreditRiskAnalyst,
    tools=[
        get_company_id_from_name,
        analyze_company_creditworthiness,
        compare_to_industry_benchmark,
        process_loan_application, # Placeholder tool
        monitor_existing_credit, # Placeholder tool
        calculate_financial_ratios,
        get_credit_profile_info,
        file_credit_ticket,
        # Removed fetch_* tools as they are likely internal helpers
        # fetch_financial_statements,
        # fetch_credit_profile,
        # fetch_risk_assessment,
    ]
)

## Run

In [44]:
# Initialize the agent

# # Example requests
# test_requests = [
#     "Analyze the creditworthiness of TechCorp Inc and provide a comprehensive risk assessment",
#     "Compare Manufacturing Solutions LLC's financial metrics against industry benchmarks",
#     "Calculate the Altman Z-Score for Green Energy Partners for the year 2024 and explain the bankruptcy risk", # Added year for calculate_financial_ratios tool
#     "Generate a full credit risk report for Healthcare Innovations including all key ratios for the past 3 years", # Specified analysis period
#     "What is the working capital trend for Regional Retail Group over the past 3 years?"
# ]

# for request in test_requests:
#     print(f"\nUser Request: {request}")
#     # Call the forward method on the initialized agent instance
#     try:
#         response = agent(user_request=request)
#         print(f"Agent Response: {response.process_result}") # Access the output field
#     except Exception as e:
#         print(f"Agent encountered an error: {e}")

#     print("-" * 80)

In [62]:
print("Testing credit risk functions...")

try:
    # Test 0: Get company_id from company name
    print("\n1. Testing get_company_id_from_name:")
    company_id = get_company_id_from_name("TechCorp Inc")
    print(f"✅ TechCorp Inc -> {company_id}")

    # Test 1: Fetch financial statements
    print("\n1. Testing fetch_financial_statements:")
    statements = fetch_financial_statements("TC001", 3)
    print(f"✅ Retrieved {len(statements)} statements for TC001")

    # Test 2: Working capital trend
    print("\n2. Testing assess_working_capital_trend:")
    wc_trend = assess_working_capital_trend(statements)
    print(f"✅ Working capital trend: {wc_trend['trend_direction']}")

    # Test 3: Risk report generation
    print("\n3. Testing generate_credit_risk_report:")
    report_id, risk_report = generate_credit_risk_report("TC001", 3)
    print(f"✅ Generated risk report: {report_id}")
    print(f"   Overall risk rating: {risk_report['overall_risk_rating']}")

    # Test 4: Company analysis
    print("\n4. Testing analyze_company_creditworthiness:")
    analysis = analyze_company_creditworthiness("TC001", 3)
    print(f"✅ Analyzed company ID TC001 creditworthiness")
    print(f"   Company name: {analysis['company_name']}")
    print(f"   Risk rating: {analysis['risk_report']['overall_risk_rating']}")

    # Test 5: Fetch financial statements
    print("\n5. Testing fetch_financial_statements:")
    result = fetch_financial_statements("TC001", 3)
    print(f"✅ Success! Retrieved {len(result)} financial statements")
    for stmt in result:
        print(f"   - Period ending: {stmt.period_end.year}-{stmt.period_end.month:02d}-{stmt.period_end.day:02d}")

    print("\n🎉 All tests passed!")

except Exception as e:
    print(f"❌ Test failed: {e}")

Testing credit risk functions...

1. Testing get_company_id_from_name:
✅ TechCorp Inc -> TC001

1. Testing fetch_financial_statements:
✅ Retrieved 3 statements for TC001

2. Testing assess_working_capital_trend:
✅ Working capital trend: improving

3. Testing generate_credit_risk_report:
✅ Generated risk report: CRC42GH0CMZ1
   Overall risk rating: low_moderate_risk

4. Testing analyze_company_creditworthiness:
✅ Analyzed company ID TC001 creditworthiness
   Company name: TechCorp Inc
   Risk rating: low_moderate_risk

5. Testing fetch_financial_statements:
✅ Success! Retrieved 3 financial statements
   - Period ending: 2024-12-31
   - Period ending: 2023-12-31
   - Period ending: 2022-12-31

🎉 All tests passed!


In [64]:
request = "Analyze the creditworthiness of TechCorp Inc and provide a comprehensive risk assessment"

response = agent(user_request=request)

In [65]:
response

Prediction(
    trajectory={'thought_0': 'The user wants to analyze the creditworthiness of "TechCorp Inc". To do this, I first need to get the company ID for "TechCorp Inc" using the `get_company_id_from_name` tool.', 'tool_name_0': 'get_company_id_from_name', 'tool_args_0': {'company_name': 'TechCorp Inc'}, 'observation_0': 'TC001', 'thought_1': 'I have successfully retrieved the company ID for TechCorp Inc, which is TC001. Now I need to analyze the creditworthiness of this company. I will use the `analyze_company_creditworthiness` tool for this purpose.', 'tool_name_1': 'analyze_company_creditworthiness', 'tool_args_1': {'company_id': 'TC001'}, 'observation_1': {'report_id': 'CRALFZ3T3V5J', 'company_id': 'TC001', 'company_name': 'TechCorp Inc', 'credit_profile': CreditProfile(company_id='TC001', credit_rating='BBB+', industry='Technology', years_in_business=8, loan_amount=2500000.0, annual_revenue=15000000.0), 'risk_report': {'report_id': 'CRALFZ3T3V5J', 'company_id': 'TC001', 'anal

### Display functions

In [70]:
def display_react_trace(trace_dict, max_width=100):
    """
    Display DSPy ReAct agent trace in a readable format for Jupyter notebooks.

    Args:
        trace_dict (dict): The trace dictionary from DSPy ReAct agent
        max_width (int): Maximum width for text wrapping
    """
    import textwrap
    from IPython.display import display, HTML
    import json

    def wrap_text(text, width=max_width):
        """Wrap text to specified width"""
        if not text:
            return ""
        return '\n'.join(textwrap.wrap(str(text), width=width))

    def safe_json_dumps(obj):
        """Safely convert object to JSON string, handling Pydantic models and other non-serializable objects"""
        try:
            # Try regular JSON serialization first
            return json.dumps(obj, indent=2)
        except TypeError:
            try:
                # If it's a Pydantic model, use model_dump()
                if hasattr(obj, 'model_dump'):
                    return json.dumps(obj.model_dump(), indent=2)
                # If it's a dict with Pydantic models, convert them
                elif isinstance(obj, dict):
                    converted_dict = {}
                    for key, value in obj.items():
                        if hasattr(value, 'model_dump'):
                            converted_dict[key] = value.model_dump()
                        elif isinstance(value, list):
                            converted_dict[key] = [item.model_dump() if hasattr(item, 'model_dump') else item for item in value]
                        else:
                            converted_dict[key] = value
                    return json.dumps(converted_dict, indent=2)
                else:
                    # Fallback to string representation
                    return str(obj)
            except Exception:
                # Final fallback
                return str(obj)

    def format_args(args_dict):
        """Format tool arguments nicely"""
        if not args_dict:
            return "None"

        formatted_args = []
        for key, value in args_dict.items():
            if isinstance(value, str) and len(value) > 50:
                # Truncate long string values
                value = value[:47] + "..."
            elif isinstance(value, dict) or hasattr(value, 'model_dump'):
                # Format dict or Pydantic model as JSON
                json_str = safe_json_dumps(value)
                value = json_str[:100] + "..." if len(json_str) > 100 else json_str
            formatted_args.append(f"  {key}: {value}")
        return '\n'.join(formatted_args)

    def format_observation(observation):
        """Format observation data (handles both strings and dicts)"""
        if isinstance(observation, dict) or hasattr(observation, 'model_dump'):
            return safe_json_dumps(observation)
        elif isinstance(observation, str):
            return observation
        else:
            return str(observation)

    def is_error_observation(observation):
        """Check if observation indicates an error"""
        try:
            if isinstance(observation, dict) or hasattr(observation, 'model_dump'):
                # Check if dict/object contains error indicators
                obs_str = safe_json_dumps(observation).lower()
                return any(error_word in obs_str for error_word in ['error', 'cannot', 'failed', 'exception'])
            elif isinstance(observation, str):
                return any(error_word in observation.lower() for error_word in ['error', 'cannot', 'failed', 'exception'])
            else:
                obs_str = str(observation).lower()
                return any(error_word in obs_str for error_word in ['error', 'cannot', 'failed', 'exception'])
        except Exception:
            # If any error occurs in checking, assume it's not an error observation
            return False

    # Extract turns from trace
    turns = []
    turn_num = 0

    while f'thought_{turn_num}' in trace_dict:
        turn = {
            'turn': turn_num,
            'thought': trace_dict.get(f'thought_{turn_num}', ''),
            'tool_name': trace_dict.get(f'tool_name_{turn_num}', ''),
            'tool_args': trace_dict.get(f'tool_args_{turn_num}', {}),
            'observation': trace_dict.get(f'observation_{turn_num}', '')
        }
        turns.append(turn)
        turn_num += 1

    # Generate HTML output for better formatting
    html_output = """
    <style>
    .react-trace {
        font-family: 'Courier New', monospace;
        background-color: #f8f9fa;
        border: 1px solid #dee2e6;
        border-radius: 8px;
        padding: 15px;
        margin: 10px 0;
    }
    .turn-header {
        background-color: #007bff;
        color: white;
        padding: 8px 12px;
        border-radius: 5px;
        margin: 10px 0 5px 0;
        font-weight: bold;
    }
    .thought-section {
        background-color: #e3f2fd;
        border-left: 4px solid #2196f3;
        padding: 10px;
        margin: 5px 0;
    }
    .tool-section {
        background-color: #f3e5f5;
        border-left: 4px solid #9c27b0;
        padding: 10px;
        margin: 5px 0;
    }
    .observation-section {
        background-color: #e8f5e8;
        border-left: 4px solid #4caf50;
        padding: 10px;
        margin: 5px 0;
    }
    .error-observation {
        background-color: #ffebee;
        border-left: 4px solid #f44336;
    }
    .section-label {
        font-weight: bold;
        color: #333;
        margin-bottom: 5px;
    }
    .section-content {
        white-space: pre-wrap;
        word-wrap: break-word;
    }
    </style>
    """

    html_output += '<div class="react-trace">'
    html_output += f'<h3>🤖 DSPy ReAct Agent Trace ({len(turns)} turns)</h3>'

    for turn in turns:
        # Turn header
        html_output += f'<div class="turn-header">Turn {turn["turn"]}</div>'

        # Thought section
        if turn['thought']:
            html_output += '<div class="thought-section">'
            html_output += '<div class="section-label">💭 Thought:</div>'
            html_output += f'<div class="section-content">{wrap_text(turn["thought"])}</div>'
            html_output += '</div>'

        # Tool section
        if turn['tool_name']:
            html_output += '<div class="tool-section">'
            html_output += '<div class="section-label">🔧 Tool Called:</div>'
            html_output += f'<div class="section-content"><strong>{turn["tool_name"]}</strong></div>'

            if turn['tool_args']:
                html_output += '<div class="section-label">📝 Arguments:</div>'
                html_output += f'<div class="section-content">{format_args(turn["tool_args"])}</div>'
            html_output += '</div>'

        # Observation section
        if turn['observation']:
            observation_class = "observation-section"
            if is_error_observation(turn['observation']):
                observation_class += " error-observation"

            html_output += f'<div class="{observation_class}">'
            html_output += '<div class="section-label">👁️ Observation:</div>'
            formatted_obs = format_observation(turn['observation'])
            html_output += f'<div class="section-content">{wrap_text(formatted_obs)}</div>'
            html_output += '</div>'

    html_output += '</div>'

    # Display the HTML
    display(HTML(html_output))

def display_react_trace_simple(trace_dict):
    """
    Simple text-based version for environments without HTML support.

    Args:
        trace_dict (dict): The trace dictionary from DSPy ReAct agent
    """
    import textwrap

    def wrap_text(text, width=80):
        if not text:
            return ""
        return '\n'.join(textwrap.wrap(str(text), width=width))

    # Extract turns
    turn_num = 0
    print("=" * 80)
    print(f"DSPy ReAct Agent Trace")
    print("=" * 80)

    while f'thought_{turn_num}' in trace_dict:
        print(f"\n🔄 TURN {turn_num}")
        print("-" * 40)

        # Thought
        thought = trace_dict.get(f'thought_{turn_num}', '')
        if thought:
            print("💭 THOUGHT:")
            print(wrap_text(thought))
            print()

        # Tool
        tool_name = trace_dict.get(f'tool_name_{turn_num}', '')
        tool_args = trace_dict.get(f'tool_args_{turn_num}', {})
        if tool_name:
            print(f"🔧 TOOL: {tool_name}")
            if tool_args:
                print("📝 ARGS:")
                for key, value in tool_args.items():
                    print(f"   {key}: {value}")
            print()

        # Observation
        observation = trace_dict.get(f'observation_{turn_num}', '')
        if observation:
            print("👁️ OBSERVATION:")
            print(wrap_text(observation))
            print()

        turn_num += 1

    print("=" * 80)

# Example usage function
def demo_trace_formatter():
    """
    Demo function showing how to use the trace formatter
    """
    # Example trace (like the one from your question)
    sample_trace = {
        'thought_0': 'The user wants to analyze the creditworthiness of TechCorp Inc and get a comprehensive risk assessment. The `analyze_company_creditworthiness` tool seems appropriate for this task.',
        'tool_name_0': 'analyze_company_creditworthiness',
        'tool_args_0': {'company_name': 'TechCorp Inc', 'analysis_years': 3},
        'observation_0': 'Cannot generate credit risk report: Insufficient financial data: only 0 years available, 3 requested',
        'thought_1': "The previous tool call failed because there was insufficient financial data. I need to try calculating financial ratios for the most recent year available.",
        'tool_name_1': 'calculate_financial_ratios',
        'tool_args_1': {'company_name': 'TechCorp Inc'},
        'observation_1': 'Successfully calculated financial ratios for TechCorp Inc',
        'thought_2': "Now I have the financial ratios. I should get the credit profile information to provide a complete analysis.",
        'tool_name_2': 'get_credit_profile_info',
        'tool_args_2': {'company_name': 'TechCorp Inc'},
        'observation_2': 'Credit Profile for TechCorp Inc:\n- Credit Rating: BBB+\n- Industry: Technology\n- Years in Business: 8\n- Requested Loan Amount: 2,500,000.00\n- Annual Revenue: 15,000,000.00'
    }

    print("Demo: HTML version (for Jupyter)")
    display_react_trace(sample_trace)

    print("\nDemo: Simple text version")
    display_react_trace_simple(sample_trace)

## Display

In [72]:
# test for formatters
#demo_trace_formatter()

display_react_trace(response["trajectory"])

In [73]:
display_react_trace_simple(response["trajectory"])

DSPy ReAct Agent Trace

🔄 TURN 0
----------------------------------------
💭 THOUGHT:
The user wants to analyze the creditworthiness of "TechCorp Inc". To do this, I
first need to get the company ID for "TechCorp Inc" using the
`get_company_id_from_name` tool.

🔧 TOOL: get_company_id_from_name
📝 ARGS:
   company_name: TechCorp Inc

👁️ OBSERVATION:
TC001


🔄 TURN 1
----------------------------------------
💭 THOUGHT:
I have successfully retrieved the company ID for TechCorp Inc, which is TC001.
Now I need to analyze the creditworthiness of this company. I will use the
`analyze_company_creditworthiness` tool for this purpose.

🔧 TOOL: analyze_company_creditworthiness
📝 ARGS:
   company_id: TC001

👁️ OBSERVATION:
{'report_id': 'CRALFZ3T3V5J', 'company_id': 'TC001', 'company_name': 'TechCorp
Inc', 'credit_profile': CreditProfile(company_id='TC001', credit_rating='BBB+',
industry='Technology', years_in_business=8, loan_amount=2500000.0,
annual_revenue=15000000.0), 'risk_report': {'report_id':

In [74]:
lm.history[-1]

{'prompt': None,
 'messages': [{'role': 'system',
   'content': "Your input fields are:\n1. `user_request` (str): \n2. `trajectory` (str):\nYour output fields are:\n1. `reasoning` (str): \n2. `process_result` (str): Detailed analysis result that summarizes the credit assessment findings, key financial metrics, risk ratings, and actionable recommendations. Include specific numbers, ratios, and risk categories. Provide report IDs, decision outcomes, or ticket numbers when applicable.\nAll interactions will be structured in the following way, with the appropriate values filled in.\n\n[[ ## user_request ## ]]\n{user_request}\n\n[[ ## trajectory ## ]]\n{trajectory}\n\n[[ ## reasoning ## ]]\n{reasoning}\n\n[[ ## process_result ## ]]\n{process_result}\n\n[[ ## completed ## ]]\nIn adhering to this structure, your objective is: \n        You are a commercial credit risk analyst that helps assess creditworthiness and manage loan applications.\n        \n        You are given a list of tools to h